<a href="https://colab.research.google.com/github/RodrigoZepeda/ShinglesDeltaMethod/blob/main/DeltaMethod_PIF_Shingles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Delta Method for Potential Impact Fraction

The following code contains the calculation necesary for estimating the variance of the potential impact fraction (PIF) with binary relative risks. It assumes the variance of the log relative risk is known as well as the variance of the proportion of individuals exposed.

In [2]:
%pip install --quiet "session_info"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.0 MB/s eta 0:00:00


## Problem definition

Let $p$ denote the probability of being exposed. Let $RR$ denote the relative risk of the exposure and $\ell = \ln RR$ the logarithm of that relative risk (no exposure has relative risk $=1$). Define:
\begin{equation}
\begin{aligned}
\mu^{\textrm{obs}} & = p\cdot \exp \big(\ln (RR)\big) + (1 - p)\cdot 1 = 1 + p\cdot (\exp(\ell) - 1)\\
\mu^{\textrm{cft}} & = 1 + p_* \cdot (\exp(\ell) - 1)\\
\end{aligned}
\end{equation}
Define the potential impact fraction as:
\begin{equation}
\text{PIF} = \dfrac{\mu^{\textrm{obs}}  - \mu^{\textrm{cft}}}{\mu^{\textrm{obs}}} = 1 - \dfrac{\mu^{\textrm{cft}}}{\mu^{\textrm{obs}}}
\end{equation}

For our scenarios we will only consider counterfactuals that **reduce** the relative risk, hence:
$$
0 <  \mu_{cft} \leq \mu_{obs}.
$$

Finally consider $\hat{p}$ an estimator of $p$ with variance $\sigma_{\hat{p}}^2$ and $\widehat{\text{RR}}$ an estimate of $\text{RR}$. Assume the variance that is known is for $\hat{\ell} = \widehat{\log \text{RR}}$ and given by $\sigma^2_{\ell}$. The plug-in estimator of the potential impact fraction, $\widehat{\text{PIF}}$, is:
\begin{equation}
\widehat{\text{PIF}} = 1 - \dfrac{\hat{\mu}^{\textrm{cft}}}{\hat{\mu}^{\textrm{obs}}}
\end{equation}
with
\begin{equation}
\begin{aligned}
\hat{\mu}^{\textrm{obs}} & = 1 + \hat{p}\cdot \big(\exp(\widehat{\ell}) - 1\big)\\
\hat{\mu}^{\textrm{cft}} & = 1 + p_* \cdot \big(\exp(\widehat{\ell}) - 1\big)\\
\end{aligned}
\end{equation}

The following Python code helps set up the expressions

In [55]:
#We'll use the sympy library for symbolic computation of variances and expected values
import sympy
from sympy import *
from sympy.stats import Variance, Expectation, Covariance
from sympy.stats.rv import RandomSymbol
from sympy import exp
from scipy.stats import norm
import session_info
import numpy as np
import pandas as pd

print(sympy.__version__)   #version of sympy should be >=1.10

#Create the constants
p = Symbol('p', real=True, positive=True)            #Proportion exposed
p_star = Symbol('p_star', real=True, positive=True)  #Proportion exposed under counterfactual
L = Symbol('L', real=True, positive=True)            #Log Relative risk of exposed (i.e. relative risk = exp(L))

#Create the random symbols for RR and p hat
p_hat = RandomSymbol('p_hat')             #Estimate of p
L_hat = RandomSymbol('L_hat')             #Estimate of log(RR)

sigma_p = Symbol('sigma_p^2', real=True, positive=True) #Variance of p_hat
sigma_l = Symbol('sigma_l^2', real=True, positive=True) #Variance of log(RR_hat)

1.13.1


## Delta method estimation

Here we construct a function that applies the delta method to any function of the potential impact fraction

In [4]:
#Potential impact fraction definition
def pif(L, p, p_star, sigma_p, sigma_l):
  mu_cft = 1 + p_star*(sympy.exp(L) - 1)
  mu_obs = 1 + p*(sympy.exp(L) - 1)
  return 1 - mu_cft/mu_obs

#Replace Cov(1, X) = 0, Cov(1, 1) = 0.0 and Var(X + constant) = Var(X)
def simplify_covariance(expr):
    expr = expr.subs(
        Covariance(1, p_hat), 0
    ).subs(
        Covariance(1, L_hat), 0
    ).subs(
        Covariance(1, 1), 0
    ).subs(
        Variance(-p + p_hat), Variance(p_hat)
    ).subs(
        Variance(-L + L_hat), Variance(L_hat)
    )
    return expr

#Replace Var(X) = sigma_X and Cov(X,Y) = 0 as p and L are independent
def expand_variance(expr):
    expr = expr.subs(
        Variance(p_hat), sigma_p
    ).subs(
        Variance(L_hat), sigma_l
    ).subs(
        Covariance(p_hat, L_hat), 0.0
    )
    return expr


def delta_method(pif_function, substitute_mu=False):
  """
  Function that applies the delta method to a function of pif

  The pif function should have at least two variables: p and L
  corresponding to the proportion infected and the log relative risk
  """

  #Obtain the direction vector
  v = Matrix([L_hat - L, p_hat - p])

  #And calculate the gradient
  grad = derive_by_array(pif_function(L, p, p_star, sigma_p, sigma_l), [L, p])
  grad = Matrix(grad)

  #Thus calculating the Hadamard (directional) derivative:
  gradient_directed = grad.dot(v)

  fun_variance = Variance(gradient_directed).expand().simplify()

  #Simplify the variance
  fun_variance = simplify_covariance(fun_variance)

  #Expand my substituting the sigmas
  fun_variance = expand_variance(fun_variance)

  #Simplify the denominator
  fun_variance = fun_variance.replace(
    fraction(fun_variance)[1],fraction(fun_variance)[1].simplify()
  )

  return fun_variance.simplify()


# Estimation of the delta method

After applying the above functions we obtain that an approximation of the variance is given by:

In [5]:
simplified_pif = delta_method(pif, False)
display(simplified_pif)

sigma_l^2*(-p + p_star)**2*exp(2*L)/(-p*exp(L) + p - 1)**4 + sigma_p^2*(1 - exp(L))**2*(-p_star*exp(L) + p_star - 1)**2/(-p*exp(L) + p - 1)**4

In [6]:
#Transform the previous results to function
fun_dm_pif = lambdify([L, p, p_star, sigma_p, sigma_l], simplified_pif)

In [47]:
#Calculate a specific PIF
def pif_calculator(L, p, p_star, sigma_p, sigma_l):
  pif_value = pif(L, p, p_star, sigma_p, sigma_l)
  variance = fun_dm_pif(L, p, p_star, sigma_p, sigma_l)
  confint = norm.ppf(0.975)*sqrt(variance)
  return 100*pif_value, 100*np.array((pif_value - confint, pif_value + confint)), variance

def case_calculator(cases, pif, sigma_cases, sigma_pif):
  variance = sigma_cases*sigma_pif + sigma_cases*pow(pif, 2) + sigma_pif*pow(cases,2)
  confint = norm.ppf(0.975)*sqrt(variance)
  return pif*cases, np.array([pif*cases - confint, pif*cases + confint]), variance

#Get SE^2 from confidence intervals
def get_var_log(UCL_RR, LCL_RR):
  return pow((log(UCL_RR) - log(LCL_RR)) / (2 * norm.ppf(0.975)),2)

def get_var(UCL_RR, LCL_RR):
  return pow((UCL_RR - LCL_RR) / (2 * norm.ppf(0.975)),2)

In [56]:
#Global values
pval    = 0.438
sigma_p_val = pow(0.0184,2)

#Attributable cases
cases = 8.53
cases_low = 8.07
cases_up = 8.99

table_data = []

#1) WU PIF
UCL_RR  = 0.72
LCL_RR  = 0.67
RR_val  = 0.69
sigma_l_val = get_var_log(UCL_RR, LCL_RR)
sigma_cases = get_var(cases_up, cases_low)
for val in [1.15, 1.30, 1.60, 1.0/pval]:
  pif_val = pif_calculator(np.log(RR_val), pval, val*pval, sigma_p_val, sigma_l_val)
  case_val = case_calculator(cases, pif_val[0]/100.0, sigma_cases, pif_val[2])
  table_data.append([
        "Wu",
        f"{100*val*pval:.2f}%",
        f"{100*(val - 1):.0f}%",
        f"{pif_val[0]:.2f}% [{pif_val[1][0]:.2f}%, {pif_val[1][1]:.2f}%]",
        f"{case_val[0]:.2f} [{case_val[1][0]:.2f}, {case_val[1][1]:.2f}]"
    ])

#2) Shah PIF
UCL_RR  = 0.96
LCL_RR  = 0.60
RR_val  = 0.76
sigma_l_val = get_var_log(UCL_RR, LCL_RR)
variance_shah = []
pif_shah = []
for val in [1.15, 1.30, 1.60, 1.0/pval]:
  pif_val = pif_calculator(np.log(RR_val), pval, val*pval, sigma_p_val, sigma_l_val)
  case_val = case_calculator(cases, pif_val[0]/100.0, sigma_cases, pif_val[2])
  table_data.append([
        "Shah",
        f"{100*val*pval:.2f}%",
        f"{100*(val - 1):.0f}%",
        f"{pif_val[0]:.2f}% [{pif_val[1][0]:.2f}%, {pif_val[1][1]:.2f}%]",
        f"{case_val[0]:.2f} [{case_val[1][0]:.2f}, {case_val[1][1]:.2f}]"
    ])


df = pd.DataFrame(table_data)
df.columns = ["Author","Vaccination coverage (%)","Increase (%)","PIF", "Prevented Cases (millions)"]
df

,Author,Vaccination coverage (%),Increase (%),PIF,Prevented Cases (millions)
0,Wu,50.37%,15%,"2.36% [1.07%, 3.64%]","0.20 [0.09, 0.31]"
1,Wu,56.94%,30%,"4.71% [3.41%, 6.02%]","0.40 [0.29, 0.52]"
2,Wu,70.08%,60%,"9.43% [7.97%, 10.89%]","0.80 [0.67, 0.94]"
3,Wu,100.00%,128%,"20.16% [18.02%, 22.29%]","1.72 [1.52, 1.92]"
4,Shah,50.37%,15%,"1.76% [0.02%, 3.51%]","0.15 [0.00, 0.30]"
5,Shah,56.94%,30%,"3.52% [0.45%, 6.60%]","0.30 [0.04, 0.56]"
6,Shah,70.08%,60%,"7.05% [1.12%, 12.98%]","0.60 [0.09, 1.11]"
7,Shah,100.00%,128%,"15.07% [2.51%, 27.63%]","1.29 [0.21, 2.36]"


#Reproducibility

In [9]:
session_info.show()